# Tournament Collection

Purpose of this notebook is to create csv files of tournaments within the Northern California area during a specific timeframe.

In [31]:
#Imports
from datetime import datetime
import json
import pandas as pd
import requests

In [32]:
AUTH_TOKEN = '3a8df0ab61dea506d92c3bab4fd27412'

In [33]:
request_url = 'https://api.start.gg/gql/alpha'

## Step 1: Retrieve Unix Timestamps for GraphQL

This allows us to batch our tournaments. 

In [34]:
def date_to_unix(date_string, date_format="%Y-%m-%d"):
    try:
        date_obj = datetime.strptime(date_string, date_format)
        unix_timestamp = int(date_obj.timestamp())
        return unix_timestamp
    except ValueError as e:
        print(f"Error: {e}")
        return None

In [35]:
def monthly_unix_timestamps(start_date, end_date, date_format="%Y-%m-%d"):
    try:
        start = datetime.strptime(start_date, date_format)
        end = datetime.strptime(end_date, date_format)
        
        if start > end:
            raise ValueError("Start date must be before the end date.")
        
        timestamps = [date_to_unix(start_date, date_format)]
        current = start
        
        while current < end:
            year = current.year
            month = current.month + 1
            if month > 12: 
                month = 1
                year += 1
            current = datetime(year, month, 1)
            if current <= end:
                timestamps.append(int(current.timestamp()))
        
        timestamps.append(date_to_unix(end_date, date_format))
        return timestamps
    except ValueError as e:
        print(f"Error: {e}")
        return None

In [36]:
start_date = "2024-10-01"
end_date = "2024-12-31"
timestamps = monthly_unix_timestamps(start_date, end_date)
timestamps

[1727766000, 1730444400, 1733040000, 1735632000]

## Step 2: Query Generation

Here, we use our batched dates to define our GraphQL Queries.

In [37]:
def generate_graphql_queries_fstring(timestamps):
    template = """
query BayNorCalTournaments($page: Int, $perPage: Int, $coordinates: String!, $radius: String!) {{
  tournaments(
    query: {{
    page: $page
    perPage: $perPage
    filter: {{
      location: {{
        distanceFrom: $coordinates,
        distance: $radius
      }},
      afterDate: {after_date} 
      beforeDate: {before_date}
    }}
    sortBy:"startAt"
  }}) {{
    nodes {{
      id
      name
      city
      slug
      startAt
      events {{
        slug
        numEntrants
        videogame {{
          name
        }}
      }}
    }}
  }}
}}
"""

    queries = []

    for i in range(len(timestamps) - 1):
        after_date = timestamps[i]
        before_date = timestamps[i + 1]

        query = template.format(after_date=after_date, before_date=before_date)
        queries.append(query)

    return queries

In [38]:
queries = generate_graphql_queries_fstring(timestamps)
print(queries[0])


query BayNorCalTournaments($page: Int, $perPage: Int, $coordinates: String!, $radius: String!) {
  tournaments(
    query: {
    page: $page
    perPage: $perPage
    filter: {
      location: {
        distanceFrom: $coordinates,
        distance: $radius
      },
      afterDate: 1727766000 
      beforeDate: 1730444400
    }
    sortBy:"startAt"
  }) {
    nodes {
      id
      name
      city
      slug
      startAt
      events {
        slug
        numEntrants
        videogame {
          name
        }
      }
    }
  }
}



## Step 3: Tournament Retrieval

From here, we will use our batches to create tournament dataframes by month. 

In [39]:
def flatten_nested_json_df(df):
    
    df = df.reset_index()
    
    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")
    
    
    # search for columns to explode/flatten
    s = (df.map(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.map(type) == dict).all()
    dict_columns = s[s].index.tolist()
    
    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []
        
        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace
        
        for col in list_columns:
            print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            # Prevent combinatorial explosion when multiple
            # cols have lists or lists of lists
            df = df.reset_index(drop=True)
            new_columns.append(col)
        
        # check if there are still dict o list fields to flatten
        s = (df[new_columns].map(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].map(type) == dict).all()
        dict_columns = s[s].index.tolist()
        
        print(f"lists: {list_columns}, dicts: {dict_columns}")
        
    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

In [40]:
def get_all_tournies(auth_token, query, coords, radius, num_per_page):
  graphql_query = query 

  tournies = []

  # This range here is used because the results from startGG are paginated, i.e. they don't show all the results,
  #   they're shown in pages. If there were 100 resuls, and we could only see 10 per page, then we would need to go through
  #   and make ten seperate requests to see all the tournaments we need
  for i in range(1, 10):
    variables = {
        "page": i,
        "perPage": num_per_page,
        "coordinates": coords,
        "radius": "50mi"
    }
    data = {"query" : graphql_query, "variables": variables}
    json_data = json.dumps(data)
    auth_header = auth_token
    header = {'Authorization': 'Bearer ' + auth_header}  


    # Extracting & making the the actual response to startgg
    response = requests.post(url=request_url, headers=header, data=json_data)
    json_resp = json.loads(response.text)
    print(json_resp)
    if ("errors" not in json_resp):
       
      curr_tournies_page = json_resp['data']['tournaments']['nodes']
      print("Number of tournies in page is:" + str(len(curr_tournies_page)))

      tournies += curr_tournies_page
  return tournies

In [41]:
tournies = []
df_list = []

SF_BASED_COORDS = "37.77151615492457, -122.41563048985462"
SF_RADIUS = "70mi"

SAC_BASED_COORDS = "38.57608096237729, -121.49183616631059"
SAC_RADIUS = "40mi"

NUM_PER_PAGE = 50

In [49]:
bay_tournies = get_all_tournies(AUTH_TOKEN, queries[1], SF_BASED_COORDS, SF_RADIUS, NUM_PER_PAGE)
sac_tournies = get_all_tournies(AUTH_TOKEN, queries[1], SAC_BASED_COORDS, SAC_RADIUS, NUM_PER_PAGE)
tournies = sac_tournies + bay_tournies
np_tournies = pd.DataFrame(tournies).explode('events')
flat_tournies = flatten_nested_json_df(np_tournies)

if 'events' not in flat_tournies.columns:
    print("Warning: 'events' column missing in flat_tournies.")
    print(flat_tournies.head())  # Debugging information
else:
    ult_tournies = flat_tournies[
        flat_tournies['events'].apply(
            lambda x: isinstance(x, dict) and x.get('videogame', {}).get('name') == 'Super Smash Bros. Ultimate'
        ) &
        flat_tournies['events'].apply(
            lambda x: isinstance(x, dict) and x.get('numEntrants', 0) >= 16
        )
    ]

ult_tournies = ult_tournies.reset_index(drop=True)

ult_tournies['startgg_url'] = ult_tournies['events'].apply(lambda x: x['slug'] if isinstance(x, dict) and 'slug' in x else None)
ult_tournies['startgg_url'] = 'start.gg/' + ult_tournies['startgg_url'].astype('str')
ult_tournies['Event Date'] = ult_tournies['startAt'].map(
    lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d') if pd.notnull(x) and isinstance(x, (int, float)) else None
)
ult_tournies['StartGG TOURNAMENT_ID'] = ult_tournies['startgg_url'].map(lambda url: url.split("/")[-3:-2][0])
ult_tournies['StartGG EVENT_ID'] = ult_tournies['startgg_url'].map(lambda url: url.split("/")[-1:][0]) 
ult_tournies = ult_tournies.drop_duplicates('startgg_url', keep='first')

ult_tournies['jakeSlug'] = ult_tournies['slug'] + '/event/' + ult_tournies['StartGG EVENT_ID']

df_list.append(ult_tournies)

{'data': {'tournaments': {'nodes': [{'id': 728412, 'name': 'Tech City Tekken #150 - T8 Weekly', 'city': 'Sunnyvale', 'slug': 'tournament/tech-city-tekken-150-t8-weekly', 'startAt': 1731036600, 'events': [{'slug': 'tournament/tech-city-tekken-150-t8-weekly/event/tekken-8-singles', 'numEntrants': 18, 'videogame': {'name': 'TEKKEN 8'}}]}, {'id': 709505, 'name': 'Astral Beatdown November 2024', 'city': 'Milpitas', 'slug': 'tournament/astral-beatdown-november-2024', 'startAt': 1731182400, 'events': [{'slug': 'tournament/astral-beatdown-november-2024/event/norcalibur', 'numEntrants': 8, 'videogame': {'name': 'SOULCALIBUR VI'}}, {'slug': 'tournament/astral-beatdown-november-2024/event/umamusume-pretty-derby-party-dash', 'numEntrants': 3, 'videogame': {'name': 'Umamusume: Pretty Derby - Party Dash'}}, {'slug': 'tournament/astral-beatdown-november-2024/event/2pm-granblue-fantasy-versus-rising', 'numEntrants': 43, 'videogame': {'name': 'Granblue Fantasy Versus: Rising'}}, {'slug': 'tournament/as

In [43]:
len(df_list)

3